# Cell-to-cell communication

Effective biological functions, such as the immune response, rely heavily on the communication between cells. When cells are dissociated for single-cell sequencing, the spatial relationships between them are disrupted, leading to loss of important information. Consequently, a number of algorithms have been devised to deduce intercellular communication from gene expression data.Typically, these techniques comprise a repository of recognized ligand-receptor pairings and a computation mechanism for assessing these pairings according to their gene expression patterns.

Dimitrov et al. {cite}`Dimitrov2022` conducted a comparative study of 16 ligand-receptor databases and 7 methods, which encompassed Cell-PhoneDB {cite}`Efremova2020`, CellChatDB {cite}`Jin2021`, and NATMI {cite}`Hou2020`. The LIANA package {cite}`Trei2021` was utilized to provide a unified interface to all methods, and they consolidated the ligand-receptor data from all databases into Omnipathdb. The researchers evaluated the methods and databases in relation to their correlation with spatial proximity and cytokine signaling, due to the abscence of a gold standard dataset that could be used for benchmarking purposes. Despite significant variations observed among both the methods and databases, no single technique emerged as consistently superior to the others. Methods like NicheNet {cite}`Browaeys2019`, cytotalk {cite}`Hu2021`, and SoptSC {cite}`Wang2019` go a step forward by also factoring in gene regulatory networks within a cell (intracellular communication). This enables the exploitation of the expression of downstream target genes of a receptor for predicting cell-cell interactions.

Initially, the techniques outlined above were developed to evaluate intercellular communication between distinct cell types. To achieve this objective, CellPhoneDB and CellChatDB deploy a permutation test that randomizes cell type labels. While this approach is valuable in comprehending the steady state across different tissues, with the single-cell research domain now embracing perturbation experiments and extensive atlases, it has become necessary to implement methods for assessing varied patterns of intercellular communication. To takle this problem we have incorporated a straightforward yet efficient technique by considering a ligand-receptor pair as potentially perturbed if there is differential expression in at least one of the ligand and receptor. By using differentially expressed ligands/receptors (from comparisons between conditions) as input, we account for pseudoreplication bias and dropouts of lowly expressed genes.

### 1. Import the required libraries

In [ ]:

import pandas as pd
import scanpy as sc

import atlas_protocol_scripts as aps

### 2. Load and sanitise the input data

In [ ]:
# Define paths
path_adata = "/home/fotakis/myScratch/atlas_tmp/atlas-integrated-annotated.h5ad"
# results_dir ="/home/fotakis/myScratch/atlas_tmp"
path_ccdb = "/home/fotakis/myScratch/atlas_tmp/differential_expression/omnipathdb.tsv"
deseq2_path_prefix = "/home/fotakis/myScratch/atlas_tmp/differential_expression/"

In [ ]:
# Load in the scRNAseq data
adata = sc.read_h5ad(path_adata)

# Load in the cellChatDB database
ccdb = pd.read_csv(path_ccdb, sep="\t")

In [ ]:
# Keep primary sites and LUAD-LUSC conditions only
adata_primary_tumor = adata[(adata.obs["origin"] == "tumor_primary") & (adata.obs["condition"] != "NSCLC NOS")].copy()

In [ ]:
# Sanity check
adata_primary_tumor.obs

### 3. Define the Immune cell types

In [ ]:
# Define immune cell types (to be used for ploting)
immune_cells = [
    "B cell",
    "cDC1",
    "cDC2",
    "DC mature",
    "Macrophage",
    "Macrophage alveolar",
    "Mast cell",
    "Monocyte",
    "Neutrophils",
    "NK cell",
    "pDC",
    "Plasma cell",
    "T cell CD4",
    "T cell CD8",
    "T cell regulatory",
]

### 4. Perform the cell-to-cell communication analysis

In [ ]:
# Sanity check
ccdb

In [ ]:
# Run the analysis function (using the receptor ligand-list and the LUAD-LUSC data)
ccdba = aps.tl._cell2cell.CpdbAnalysis(
    ccdb,
    adata_primary_tumor,
    pseudobulk_group_by=["patient"],
    cell_type_column="cell_type_major",
)

### 5. Use the DE results to infer cell-to-cell communications

In [ ]:
# Load in DE results (using LUAD as reference)
de_res_tumor_cells_luad_lusc = (
    pd.read_csv(
        (deseq2_path_prefix + "/IHWallGenes.tsv").format(comparison="luad_lusc"),
        sep=",",
    )
    .fillna(1)
    .pipe(aps.tl._fdr.fdr_correction)
    .assign(group="LUAD")
)

In [ ]:
# Sanity check
de_res_tumor_cells_luad_lusc

In [ ]:
# Save the results
ccdb_res = ccdba.significant_interactions(de_res_tumor_cells_luad_lusc, max_pvalue=0.1)
# ccdb_res.to_csv(f"{results_dir}/cpdb_luad_lusc_coarse.csv")

### 6. Visualisation

In [ ]:
# Load in the results
ccdb_res = ccdba.significant_interactions(de_res_tumor_cells_luad_lusc, max_pvalue=0.1)

In [ ]:
# Keep only the immune cells (we'll use this for plotting)
ccdb_res = ccdb_res.loc[lambda x: x["cell_type_major"].isin(immune_cells)]

In [ ]:
# Create a list of unique top-upregulated genes
top_genes = (
    ccdb_res.loc[:, ["source_genesymbol", "fdr"]]
    .drop_duplicates()
    .sort_values("fdr")["source_genesymbol"][:30]
    .tolist()
)

In [ ]:
# Plot the results
ccdba.plot_result(
    ccdb_res.loc[lambda x: x["source_genesymbol"].isin(top_genes)],
    title="LUAD vs LUSC: tumor cells, top 30 DE ligands",
    aggregate=False,
    cluster="heatmap",
    label_limit=80,
)